In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ipyleaflet import AwesomeIcon, Map, Marker, MarkerCluster, SplitMapControl
from geopy.geocoders import Nominatim


In [2]:
c_data = pd.read_csv("covid19_cleaned_8_10.zip")
countries = pd.read_csv("countries.csv")

C:\Users\dpale\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9,10,12,13,14,15,17,18,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
c_data.columns

Index(['Unnamed: 0', 'ID', 'age', 'sex', 'city', 'province', 'country',
       'latitude', 'longitude', 'date_onset_symptoms',
       'date_admission_hospital', 'date_confirmation', 'symptoms',
       'travel_history_dates', 'travel_history_location',
       'reported_market_exposure', 'chronic_disease_binary', 'chronic_disease',
       'outcome', 'date_death_or_discharge', 'admin3', 'admin2', 'admin1',
       'travel_history_binary', 'hospitalized', 'deceased', 'fever', 'cough',
       'fatigue', 'headache', 'dizziness', 'sore throat', 'pneumonia',
       'respiratory', 'nausea', 'diarrhea', 'severe_r'],
      dtype='object')

In [4]:
locations = c_data[['latitude', 'longitude', "country", "city", "travel_history_binary"]]
locations = locations.dropna()

In [5]:
locations["travel_history_binary"].value_counts()

0    1425997
1      14267
Name: travel_history_binary, dtype: int64

In [ ]:
len(countries.loc[countries["name"] == "Uniteasd States", "latitude"])

In [ ]:
def applyLat(c):
    if len(countries.loc[countries["name"] == c, "latitude"]) == 0:
        return 0
    return countries.loc[countries["name"] == c, "latitude"].values[0]

def applyLong(c):
    if len(countries.loc[countries["name"] == c, "longitude"]) == 0:
        return 0
    return countries.loc[countries["name"] == c, "longitude"].values[0]
locations["country_lat"] =  locations["country"].apply(applyLat)
locations["country_long"] =  locations["country"].apply(applyLong)


In [5]:
locations

,latitude,longitude,country,city,travel_history_binary
0,22.365019,114.133808,China,Shek Lei,0
1,45.297748,11.658382,Italy,Vo Euganeo,0
3,34.629310,113.468000,China,Zhengzhou City,0
4,27.513560,113.902900,China,Pingxiang City,0
5,28.307550,114.973200,China,Yichun City,0
...,...,...,...,...,...
2309941,52.302440,-0.696320,United Kingdom,"Wellingborough,Northamptonshire",0
2309952,51.400900,-1.323500,United Kingdom,"West Berkshire,Berkshire",0
2309963,53.600263,-2.870291,United Kingdom,"West Lancashire,Lancashire",0
2309974,51.837658,-1.506852,United Kingdom,"West Oxfordshire,Oxfordshire",0


In [6]:
locations.dtypes

latitude                 float64
longitude                float64
country                   object
city                      object
travel_history_binary      int64
dtype: object

In [7]:
locationsUS = locations[locations["country"] == "United States"]

In [8]:
locationsUS["latitude"].value_counts()

40.661000    20325
41.162492     4679
40.739947     3241
40.876838     2058
47.491332     1666
             ...  
47.737285        1
46.084525        1
43.987557        1
29.320694        1
41.773382        1
Name: latitude, Length: 1272, dtype: int64

In [9]:
latitudes = locationsUS["latitude"].to_numpy()
longitudes = locationsUS["longitude"].to_numpy()
circle = AwesomeIcon(
    name="fa-circle"
)
pairs = [Marker(icon=circle, location=(lat, long)) for lat, long in zip(latitudes, longitudes)]

In [14]:
marker_cluster = MarkerCluster(
    markers = pairs
)

In [12]:
m = Map(center=((-29.9216045, -122.53391)), zoom=5)
m.add_layer(marker_cluster)

In [13]:
display(m)

Map(center=[-29.9216045, -122.53391], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

In [15]:
# Only the cases in the US that traveled
onlyTraveled = locationsUS[locationsUS["travel_history_binary"] == 1]

In [16]:
t_latitudes = onlyTraveled["latitude"].to_numpy()
t_longitudes = onlyTraveled["longitude"].to_numpy()
circle = AwesomeIcon(
    name="fa-circle"
)
t_pairs = [Marker(icon=circle, location=(lat, long)) for lat, long in zip(t_latitudes, t_longitudes)]

In [17]:
t_pairs

[Marker(icon=AwesomeIcon(name='fa-circle'), location=[41.2955677, -96.153827], options=['alt', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'rotation_angle', 'rotation_origin', 'title', 'z_index_offset']),
 Marker(icon=AwesomeIcon(name='fa-circle'), location=[41.2955677, -96.153827], options=['alt', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'rotation_angle', 'rotation_origin', 'title', 'z_index_offset']),
 Marker(icon=AwesomeIcon(name='fa-circle'), location=[41.2955677, -96.153827], options=['alt', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'rotation_angle', 'rotation_origin', 'title', 'z_index_offset']),
 Marker(icon=AwesomeIcon(name='fa-circle'), location=[41.2955677, -96.153827], options=['alt', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'rotation_angle', 'rotation_origin', 'title', 'z_index_offset']),
 Marker(icon=AwesomeIcon(name='fa-circle'), location=[60.379254, 149.68531299999995], options=['alt', 'draggable', 'keyboard', '

In [18]:
travel_cluster = MarkerCluster(
    markers = t_pairs
)

In [22]:
m2 = Map(center=((-29.9216045, -122.53391)), zoom=5)
m2.add_layer(travel_cluster)


In [23]:
display(m2)

Map(center=[-29.9216045, -122.53391], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…